In [1]:
import os
import json
from keras import Sequential
from keras.layers import Lambda, Dense
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras.backend as K
import numpy as np

from utility.train_data_loader import load_train_data


Using TensorFlow backend.


In [2]:
data_root = "../../beauty_image/"


In [3]:
categories_file = open("../data/categories.json", "r")
categories = json.load(categories_file)

In [4]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [5]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2"


In [6]:
trainData = load_train_data()


custom train data used


In [7]:
train_data_specialized = trainData[trainData['image_path'].str.contains("beauty")]
train_data_specialized['image_path'] = train_data_specialized['image_path'].map(lambda x: x.lstrip('beauty_image/'))

C:\Users\Hans Tananda\.virtualenvs\ndsc-beginner\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
categories_specialized = {k.lower(): v for k, v in categories['Beauty'].items()}


In [9]:
IMAGE_SIZE = hub.get_expected_image_size(hub.Module(feature_extractor_url))


INFO:tensorflow:Using C:\Users\HANSTA~1\AppData\Local\Temp\tfhub_modules to cache modules.


In [10]:
image_data = image_generator.flow_from_dataframe(train_data_specialized,
                                                 directory=os.path.join(data_root),
                                                 x_col="image_path",
                                                 y_col="item_category",
                                                 target_size=IMAGE_SIZE,
                                                 color_mode="rgb",
                                                 class_mode="categorical",
                                                 shuffle=True,
                                                 )


Found 232873 images belonging to 17 classes.


In [11]:
print(image_data)

In [12]:
label_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
label_names = np.array([key.title() for key, value in label_names])


In [13]:
for image_batch, label_batch in image_data:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 17)


In [13]:
def feature_extractor(x):
    feature_extractor_module = hub.Module(feature_extractor_url)
    return feature_extractor_module(x)

In [14]:
for image_batch, label_batch in image_data:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break


Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 17)


In [24]:
model = Sequential()
model.add(Lambda(feature_extractor, input_shape=IMAGE_SIZE+[3], trainable=True))
model.add(Dense(len(categories_specialized), activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                21777     
Total params: 21,777
Trainable params: 21,777
Non-trainable params: 0
_________________________________________________________________


In [14]:
sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)
result = model.predict(image_batch)
labels = label_names[np.argmax(result, axis=-1)]

NameError: name 'model' is not defined

In [44]:
labels

array(['Foundation', 'Powder', 'Lipstick', 'Foundation', 'Concealer',
       'Highlighter', 'Setting Spray', 'Face Palette', 'Powder',
       'Foundation', 'Foundation', 'Powder', 'Foundation', 'Highlighter',
       'Foundation', 'Highlighter', 'Highlighter', 'Foundation',
       'Foundation', 'Foundation', 'Foundation', 'Highlighter',
       'Foundation', 'Foundation', 'Face Palette', 'Foundation',
       'Foundation', 'Foundation', 'Powder', 'Powder', 'Foundation',
       'Foundation'], dtype='<U20')

In [35]:
print(result.shape)

(32, 17)


In [ ]:
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [26]:
steps_per_epoch = image_data.samples//image_data.batch_size
batch_stats = CollectBatchStats()


In [15]:
test = np.array(item for item in image_data)

In [16]:
print(test.shape)

()


In [ ]:
model.fit(np.array(image_data), epochs=epochs,
          steps_per_epoch=steps_per_epoch,
          callbacks=[checkpointer],
          )